In [ ]:
! pip install transformers peft evaluate
! pip install -- upgrade datasets

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
import os
import argparse
import torch
import evaluate
import torch.nn as nn
from tqdm import tqdm
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import AutoModelForSequenceClassification,AutoTokenizer,get_linear_schedule_with_warmup,set_seed
from peft import get_peft_config,get_peft_model,get_peft_model_state_dict,set_peft_model_state_dict,PeftType,PrefixTuningConfig,PromptEncoderConfig

In [8]:
batch_size=32
model_name='roberta-large'
task='mrpc'
peft_type=PeftType.P_TUNING
device="cuda"
num_epochs=20

In [7]:
# Adding 20 learnable tokens to the input.
# Use small MLP to generate tokens dynamically
peft_config=PromptEncoderConfig(task_type='SEQ_CLS',num_virtual_tokens=20,encoder_hidden_size=128)
lr=1e-3

In [9]:
if any(k in model_name for k in ("gpt", "opt", "bloom")):
    padding_side = "left"
else:
    padding_side = "right"

tokenizer=AutoTokenizer.from_pretrained(model_name,padding_side=padding_side)
if getattr(tokenizer,'pad_token_id') is None:
  tokenizer.pad_token_id=tokenizer.eos_token_id

datasets=load_dataset('glue',task)
metric=evaluate.load('glue',task)

def tokenize_function(examples):
  outputs=tokenizer(examples['sentence1'],examples['sentence2'],truncation=True,max_length=None)
  return outputs


tokenized_datasets = datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=["idx", "sentence1", "sentence2"],
)

# Rename
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")


def collate_fn(examples):
    return tokenizer.pad(examples, padding="longest", return_tensors="pt")


# Instantiate dataloaders.
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, collate_fn=collate_fn, batch_size=batch_size)
eval_dataloader = DataLoader(tokenized_datasets["validation"], shuffle=False, collate_fn=collate_fn, batch_size=batch_size)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [11]:
# Creating PEFT in LLM
model=AutoModelForSequenceClassification.from_pretrained(model_name,return_dict=True)
model=get_peft_model(model,peft_config)
model.print_trainable_parameters()
model

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 300,288 || all params: 355,662,082 || trainable%: 0.0844


PeftModelForSequenceClassification(
  (base_model): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 1024, padding_idx=1)
        (position_embeddings): Embedding(514, 1024, padding_idx=1)
        (token_type_embeddings): Embedding(1, 1024)
        (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-23): 24 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSdpaSelfAttention(
                (query): Linear(in_features=1024, out_features=1024, bias=True)
                (key): Linear(in_features=1024, out_features=1024, bias=True)
                (value): Linear(in_features=1024, out_features=1024, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): Robe

In [12]:
# Optimizer and Learnable LR
optimizer = AdamW(params=model.parameters(),lr=lr)

lr_scheduler=get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader)* num_epochs)
)

In [13]:
model.to(device)
for epoch in range(num_epochs):
    model.train()
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch.to(device)
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch.to(device)
        with torch.no_grad():
            outputs = model(**batch)
        predictions = outputs.logits.argmax(dim=-1)
        predictions, references = predictions, batch["labels"]
        metric.add_batch(
            predictions=predictions,
            references=references,
        )

    eval_metric = metric.compute()
    print(f"epoch {epoch}:", eval_metric)

100%|██████████| 13/13 [00:07<00:00,  1.78it/s]


epoch 0: {'accuracy': 0.5196078431372549, 'f1': 0.6273764258555133}


100%|██████████| 13/13 [00:07<00:00,  1.71it/s]


epoch 1: {'accuracy': 0.6862745098039216, 'f1': 0.8083832335329342}


100%|██████████| 13/13 [00:07<00:00,  1.70it/s]


...

epoch 17: {'accuracy': 0.6936274509803921, 'f1': 0.8164464023494861}


100%|██████████| 13/13 [00:07<00:00,  1.73it/s]


epoch 18: {'accuracy': 0.696078431372549, 'f1': 0.8176470588235294}


100%|██████████| 13/13 [00:07<00:00,  1.71it/s]

epoch 19: {'accuracy': 0.696078431372549, 'f1': 0.8176470588235294}


In [15]:
# Push to HUB
model.push_to_hub('OmBhandwalkar/roberta-large-peft-p-tuning',use_auth_token=True)

/usr/local/lib/python3.11/dist-packages/transformers/utils/hub.py:920: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/82.0k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/OmBhandwalkar/roberta-large-peft-p-tuning/commit/41c92c78f1df328abfc458cb47a2052c64010904', commit_message='Upload model', commit_description='', oid='41c92c78f1df328abfc458cb47a2052c64010904', pr_url=None, repo_url=RepoUrl('https://huggingface.co/OmBhandwalkar/roberta-large-peft-p-tuning', endpoint='https://huggingface.co', repo_type='model', repo_id='OmBhandwalkar/roberta-large-peft-p-tuning'), pr_revision=None, pr_num=None)

In [16]:
# Load Adapter from HUB

import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "OmBhandwalkar/roberta-large-peft-p-tuning"
config = PeftConfig.from_pretrained(peft_model_id)
inference_model = AutoModelForSequenceClassification.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
inference_model = PeftModel.from_pretrained(inference_model, peft_model_id)

inference_model.to(device)
inference_model.eval()
for step, batch in enumerate(tqdm(eval_dataloader)):
    batch.to(device)
    with torch.no_grad():
        outputs = inference_model(**batch)
    predictions = outputs.logits.argmax(dim=-1)
    predictions, references = predictions, batch["labels"]
    metric.add_batch(
        predictions=predictions,
        references=references,
    )

eval_metric = metric.compute()
print(eval_metric)

adapter_config.json:   0%|          | 0.00/431 [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


adapter_model.safetensors:   0%|          | 0.00/82.0k [00:00<?, ?B/s]

100%|██████████| 13/13 [00:06<00:00,  1.91it/s]

{'accuracy': 0.3161764705882353, 'f1': 0.0}
